# EDA Basic Lesson

As usual, we import the necessary libraries.

In [840]:
import pandas as pd
import numpy as np

## Summarizing and Computing Descriptive Statistics

> **📊 Visual Illustration Available**: See `illustrations/01_descriptive_statistics.png` for visual representations of descriptive statistics including histograms, box plots, and how statistics like mean, median, and quartiles relate to data distributions.


`Series` and `DataFrame` have a number of common mathematical and statistical methods. Most of these fall into the category of _reductions_ or _summary statistics_, methods that extract a single value (like the sum or mean) from a Series or a Series of values from the rows or columns of a DataFrame. They have built-in handling for missing data.

In [841]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"], columns=["one", "two"])

In [842]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


As always, the default `axis` is `axis=0` for `DataFrame` methods, which means that by default, the reduction takes place over rows.

The method returns a Series containing column sums.

In [843]:
df.sum()

one    9.25
two   -5.80
dtype: float64

In [844]:
# passing axis='columns' or axis=1 sums over the columns instead
df.sum(axis="columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

When an entire row or column contains only missing values, the sum is 0, whereas if any value is NA, it will be skipped. This can be disabled with the `skipna` option.

In [845]:
df.sum(skipna=False)

one   NaN
two   NaN
dtype: float64

In [846]:
df.sum(axis=1, skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

Some methods like `idxmin` and `idxmax`, return indirect statistics like the index value where the minimum or maximum values are attained:

In [847]:
df.idxmax()

one    b
two    d
dtype: object

In [848]:
df.idxmin()

one    d
two    b
dtype: object

Other methods are accumulations (as opposed to reductions):

In [849]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Refer to the [API reference](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#computations-descriptive-stats) for a list of methods.

> Compute the mean across rows and columns.

In [850]:
df.mean()

one    3.083333
two   -2.900000
dtype: float64

In [851]:
df.mean(axis=1)

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

`describe` produces multiple summary statistics in one shot:

In [852]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


It produces different summary statistics for non-numerical data.

In [853]:
obj = pd.Series(["c", "a", "d", "a", "b", "b", "c", "c"])

obj.describe()

count     8
unique    4
top       c
freq      3
dtype: object

`unique` returns a Numpy array of unique values in a `Series`.

In [854]:
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

You can also use `value_counts` to return a `Series` containing unique values as its index and frequencies as its values, ordered count in descending order.

In [855]:
obj.value_counts()

c    3
a    2
b    2
d    1
dtype: int64

You can also sort the result by index.

In [856]:
obj.value_counts().sort_index()

a    2
b    2
c    3
d    1
dtype: int64

> Sort value counts in ascending order.

In [857]:
obj.value_counts().sort_index(ascending=True)

a    2
b    2
c    3
d    1
dtype: int64

## Handling Missing Data

> **📊 Visual Illustration Available**: See `illustrations/03_missing_data.png` for visualizations of missing data patterns, heatmaps, and comparison of different strategies for handling missing values.


Missing data is common in most data analysis applications. One of the goals in designing pandas was to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The `isna` method gives us a Boolean Series with `True` where values are null:

In [858]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [859]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python `None` value is also treated as NA:

In [860]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [861]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

You can use `dropna` to filter out missing data. On a Series, it returns the Series with only the non-null data and index values:

In [862]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [863]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [864]:
# which is equivalent to this

data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns which are all NA or just those containing any NAs. `dropna` by default drops any row containing a missing value:

In [865]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], 
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [866]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA:

In [867]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1` or `axis='columns'`:

In [868]:
# first let's add a column of all NaNs

data[4] = np.nan

data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [869]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the `thresh` argument:

In [870]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
# set some missing values
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan

df

,0,1,2
0,1.007189,NaN,NaN
1,0.228913,NaN,NaN
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [871]:
df.dropna()

,0,1,2
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [872]:
df.dropna(thresh=2)

,0,1,2
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


Rather than filtering out missing data, you may want to fill in the "holes" in any number of ways. Calling `fillna` with a constant replaces missing values with that value:

In [873]:
df.fillna(0)

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


You can also specify different fill values for each column by passing a dict.

In [874]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,1.007189,0.500000,0.000000
1,0.228913,0.500000,0.000000
2,-2.001637,0.500000,1.669025
3,-0.438570,0.500000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


The same interpolation methods available for reindexing can be used with `fillna`, but instead of forward filling, we will try backward filling.

In [875]:
df.fillna(method="bfill")

,0,1,2
0,1.007189,-1.021228,1.669025
1,0.228913,-1.021228,1.669025
2,-2.001637,-1.021228,1.669025
3,-0.438570,-1.021228,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [876]:
df.fillna(method="bfill", limit=2)

,0,1,2
0,1.007189,NaN,1.669025
1,0.228913,NaN,1.669025
2,-2.001637,-1.021228,1.669025
3,-0.438570,-1.021228,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


A common replacement value is the mean or median statistics:

In [877]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [878]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

In [879]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[[2,4,5], 1] = np.nan
df.iloc[4:, 2] = np.nan
df.iloc[3:5, 0] = np.nan

df

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,NaN,0.332883
3,NaN,-0.199543,-1.541996
4,NaN,NaN,NaN
5,0.377984,NaN,NaN


> Handle the missing values in df with the following methods:
> 1. Remove rows with any missing values.
> 2. Remove rows with all missing values.
> 3. Fill missing values with forward fill.
> 4. Fill missing values with mean of the column.

In [880]:
df.dropna()

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934


In [881]:
df.dropna(how="all")

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,NaN,0.332883
3,NaN,-0.199543,-1.541996
5,0.377984,NaN,NaN


In [882]:
df.fillna(method="ffill")

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,0.119827,-0.199543,-1.541996
4,0.119827,-0.199543,-1.541996
5,0.377984,-0.199543,-1.541996


In [883]:
df.fillna(df.mean())

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,-0.669876,0.332883
3,-0.298525,-0.199543,-1.541996
4,-0.298525,-0.669876,-1.083952
5,0.377984,-0.669876,-1.083952


## Handling Duplicates

> **📊 Visual Illustration Available**: See `illustrations/06_duplicates.png` for visualizations showing how duplicates affect datasets and the impact of different `keep` parameter options.


Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [884]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The dataframe method `duplicated` returns a boolean Series indicating whether each row is a duplicate or not:

In [885]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates` returns a DataFrame where the duplicated array is `False`:

In [886]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


You can specify any subset of columns to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [887]:
data["v1"] = range(7)

data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [888]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicate` and `drop_duplicates` by default keep the first observed value combination. Passing `keep='last'` will return the last one:

In [889]:
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data

A common step in data cleaning is transforming data based on the values in an array, Series, or column in a DataFrame.

In [890]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "pastrami", 
                                "corned beef", "bacon", "pastrami", "honey ham", 
                                "nova lox"],
                         "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you want to add a column indicating the animal type that each food came from. Let's map the food type to the animal type by creating a dictionary:

In [891]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}

The `map` method on a Series accepts a function or dict-like object containing a mapping.

In [892]:
data["animal"] = data["food"].map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Passing a function works too:

In [893]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal) ## a functiona that applies to each element of the Series

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

`replace` provides a more generic way to do element-wise replacement:

In [894]:
data = pd.Series([1, -999, 2, -999, -1000, 3])

data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

The `-999` values might be sentinel values for missing data. To replace these with NA values:

In [895]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

You can also replace multiple values at once by passing a list:

In [896]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes or a dict:

In [897]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [898]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [899]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[2:, 1] = -999
df.iloc[4:, 2] = 999

df

,0,1,2
0,1.349742,0.069877,0.246674
1,-0.011862,1.004812,1.327195
2,-0.919262,-999.000000,0.022185
3,0.758363,-999.000000,0.862580
4,-0.010032,-999.000000,999.000000
5,0.852965,-999.000000,999.000000


> Replace -999 with NaN and 999 with 0.

In [900]:
df.replace({-999: np.nan, 999: 0})

,0,1,2
0,1.349742,0.069877,0.246674
1,-0.011862,1.004812,1.327195
2,-0.919262,NaN,0.022185
3,0.758363,NaN,0.862580
4,-0.010032,NaN,0.000000
5,0.852965,NaN,0.000000


## Renaming Axis Indexes

Axis labels (index or columns) can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.

In [901]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [902]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


You can use the `map` method like a Series.

In [903]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Assigning back to the index will modify the DataFrame in-place:

In [904]:
data.index = data.index.map(transform)

In [905]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


You can also use the `rename` method which accepts a function or dict-like object providing a mapping.

In [906]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [907]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": 3})

,one,two,3,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Handling Outliers

> **📊 Visual Illustration Available**: See `illustrations/04_outliers.png` for visualizations of outlier detection methods (IQR, Z-score) and comparison of different handling strategies.


Consider a DataFrame with some normally distributed data:

In [908]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))

data

,0,1,2,3
0,-2.304234,-0.652469,-1.218302,-1.332610
1,1.074623,0.723642,0.690002,1.001543
2,-0.503087,-0.622274,-0.921169,-0.726213
3,0.222896,0.051316,-1.157719,0.816707
4,0.433610,1.010737,1.824875,-0.997518
...,...,...,...,...
995,1.395191,-0.712856,0.620955,-0.490551
996,-1.416465,0.232588,2.162370,-0.703736
997,-0.703728,0.811639,0.684245,-1.338431
998,1.880710,1.328141,-0.348691,-0.553122


In [909]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.004111,-0.052556,0.071676,0.013681
std,0.998961,0.993721,0.993230,1.003396
min,-3.745356,-3.428254,-3.548824,-3.184377
25%,-0.633050,-0.758970,-0.588138,-0.660924
50%,-0.010997,-0.111176,0.098240,-0.008947
75%,0.662336,0.629347,0.780282,0.674685
max,3.927528,3.366626,2.653656,3.260383


Let's assume that values exceeding 3 in absolute value are outliers (the threshold value can be adjusted accordingly). To find the values in one of the columns exceeding 3 or -3:

In [910]:
col = data[2]

col[col.abs() > 3]


791   -3.548824
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or -3, you can use the `any` method on a Boolean DataFrame:

In [911]:
# The parentheses around data.abs() > 3 are necessary in order to call the any method on the result of the comparison operation
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
81,3.927528,-0.255126,0.854137,-0.364807
85,0.194788,-0.655054,-0.565230,3.176873
289,-3.399312,-0.974657,-0.685312,-0.645858
307,-0.817649,0.050188,1.951312,3.260383
384,-3.745356,-1.520113,-0.346839,-0.696918
482,-1.341493,-0.293333,-0.242459,-3.056990
506,0.425384,-3.428254,-0.296336,-0.439938
569,-0.085420,0.275144,1.179227,-3.184377
791,-0.150923,-0.362528,-3.548824,1.553205
883,1.397822,3.366626,-2.372214,0.851010


To deal with outliers, you can cap them to a maximum or minimum value. Here `np.sign` returns an array of 1 and -1 depending on the sign of the values:

In [912]:
data[data.abs() > 3] = np.sign(data) * 3

In [913]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.004328,-0.052495,0.072225,0.013485
std,0.991917,0.991159,0.991378,1.001298
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.633050,-0.758970,-0.588138,-0.660924
50%,-0.010997,-0.111176,0.098240,-0.008947
75%,0.662336,0.629347,0.780282,0.674685
max,3.000000,3.000000,2.653656,3.000000


Another common way to deal with outliers is to drop rows that contain them.

In [914]:
data[(data.abs() < 3).all(axis="columns")]

,0,1,2,3
0,-2.304234,-0.652469,-1.218302,-1.332610
1,1.074623,0.723642,0.690002,1.001543
2,-0.503087,-0.622274,-0.921169,-0.726213
3,0.222896,0.051316,-1.157719,0.816707
4,0.433610,1.010737,1.824875,-0.997518
...,...,...,...,...
995,1.395191,-0.712856,0.620955,-0.490551
996,-1.416465,0.232588,2.162370,-0.703736
997,-0.703728,0.811639,0.684245,-1.338431
998,1.880710,1.328141,-0.348691,-0.553122


> Why `all` instead of `any`?
>
> Change the cap values to 2 and -2 instead.

In [915]:
data[(data.abs() < 2).all(axis="columns")]

,0,1,2,3
1,1.074623,0.723642,0.690002,1.001543
2,-0.503087,-0.622274,-0.921169,-0.726213
3,0.222896,0.051316,-1.157719,0.816707
4,0.433610,1.010737,1.824875,-0.997518
5,0.850591,-0.131578,0.912414,0.188211
...,...,...,...,...
994,-0.450105,1.543588,-0.174925,-1.766967
995,1.395191,-0.712856,0.620955,-0.490551
997,-0.703728,0.811639,0.684245,-1.338431
998,1.880710,1.328141,-0.348691,-0.553122


## Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [916]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))

df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [917]:
sampler = np.random.permutation(5)

sampler

array([3, 0, 1, 2, 4])

You can then use `iloc` indexing or `take` method to reorder your data based on the randomly generated integer array:

In [918]:
df.iloc[sampler]

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34


In [919]:
df.take(sampler)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
4,28,29,30,31,32,33,34


Permuting columns work too:

In [920]:
column_sampler = np.random.permutation(df.shape[1])

column_sampler

array([4, 2, 0, 1, 3, 5, 6])

In [921]:
df.take(column_sampler, axis=1)

,4,2,0,1,3,5,6
0,4,2,0,1,3,5,6
1,11,9,7,8,10,12,13
2,18,16,14,15,17,19,20
3,25,23,21,22,24,26,27
4,32,30,28,29,31,33,34


To select a random subset without replacement (the same row cannot appear twice), you can use the `sample` method on Series and DataFrame:

In [922]:
df.sample(n=3)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20


You can also sample with replacement (allowing repeat choices), by passing `replace=True`:

In [923]:
choices = pd.Series([5, 7, -1, 6, 4])

choices.sample(n=10, replace=True)

4    4
2   -1
3    6
0    5
1    7
0    5
0    5
4    4
2   -1
3    6
dtype: int64

> Sample `df` using the parameter `frac` instead of `n`.

In [924]:
df.sample(frac=3, replace=True)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34
4,28,29,30,31,32,33,34
4,28,29,30,31,32,33,34


## String Manipulation

Cleaning up a messy dataset for analysis often requires a lot of string manipulation. To complicate matters, a column containing strings will sometimes have missing data:

In [925]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

String and regular expression methods can be applied (passing a lambda or other function) to each value using `map`, but it will fail on the NA values. To cope with this, Series has array-oriented methods for string operations that skip NA values. These are accessed through Series's `str` attribute.

In [926]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note that the result of this operation has an `object` dtype. 

Pandas has `extension types` that provide for specialized treatment of strings, integers and boolean data. We discussed `StringDType` in the previous lesson. Refer to the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes) for more information.

In [927]:
data_as_string = data.astype('string')

data_as_string

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [928]:
data_as_string.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

You can also slice strings using this syntax:

In [929]:
data_as_string.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes       <NA>
dtype: string

Regular expressions can be used, too, along with any `re` options like `IGNORECASE`:

In [930]:
import re

In [931]:
# regex email pattern
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use `str.get` or index into the `str` attribute:

In [932]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]

matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [933]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

The `extract` method will return the captured groups of a regular expression as a DataFrame:

In [934]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


Refer to the [API reference](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling) for a list of string methods.

> Get the 1st group of the regex email pattern.
>
> Convert data into titlecase.

In [935]:
matches.str.get(0).str.title()


Dave      Dave
Steve    Steve
Rob        Rob
Wes        NaN
dtype: object

## Categorical Data

> **📊 Visual Illustration Available**: See `illustrations/05_categorical_binning.png` for visualizations comparing `pd.cut()` vs `pd.qcut()`, categorical operations, and dummy variables.


Pandas has a `categorical` type for string values with a smaller set of distinct values that are repeated. Consider the following example:

In [936]:
values = pd.Series(["apple", "orange", "apple", "apple"] * 2)

values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [937]:
values.unique()

array(['apple', 'orange'], dtype=object)

In [938]:
values.value_counts()

apple     6
orange    2
dtype: int64

Many data systems (for data warehousing, statistical computing, or other uses) have developed specialized approaches for representing data with repeated values for more efficient storage and computation. In data warehousing, a best practice is to use so-called dimension tables containing the distinct values and storing the primary observations as `integer keys` referencing the dimension table:

In [939]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(["apple", "orange"])

values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [940]:
dim

0     apple
1    orange
dtype: object

We can use `take` to restore the original Series of strings:

In [941]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

This representation as integers is called the _category codes_ or simply _codes_. The array of distinct values can be called the _categories_.

The categorical representation can yield significant performance improvements when you are doing analytics. You can also perform transformations on the categories while leaving the codes unmodified.

Pandas' `categorical` type holds data that uses the integer-based categorical representation or encoding.

In [942]:
fruits = ["apple", "orange", "apple", "apple"] * 2
N = len(fruits)

# to ensure reproducibility
rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({"fruit": fruits, 
                   "basket_id": np.arange(N), 
                   "count": rng.integers(3, 15, size=N), 
                   "weight": rng.uniform(0, 4, size=N)}, 
                  columns=["basket_id", "fruit", "count", "weight"])

df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [943]:
fruit_cat = df['fruit'].astype('category')

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

The values for `fruit_cat` are now an instance of `pandas.Categorical`, which you can access via the `.array` attribute:

In [944]:
c = fruit_cat.array

type(c)

pandas.core.arrays.categorical.Categorical

In [945]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [946]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

A trick to get mapping between codes and categories is:

In [947]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [948]:
df['fruit'] = df['fruit'].astype('category')

df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


If you have obtained categorical encoded data from another source, you can use the `pd.Categorical.from_codes` constructor:

In [949]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats = pd.Categorical.from_codes(codes, categories)

my_cats

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

Unless explicitly specified, categorical conversions assume no specific ordering of the categories. So the `categories` array may be in a different order depending on the ordering of the input data. When using `from_codes` or any of the other constructors, you can indicate that the categories have a meaningful ordering:

In [950]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cats

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

The output `[foo < bar < baz]` indicates that `'foo'` precedes `'bar'` in the ordering, and so on. 

### Computations with Categoricals

Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [951]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Assuming you want to divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you can use `cut`:

In [952]:
bins = [18, 25, 35, 60, 100]

age_cat = pd.cut(ages, bins)

age_cat

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object returned is `categorical`. Each bin is identified by a special (unique to pandas) `interval` value type containing the lower and upper limit of each bin:

In [953]:
age_cat.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [954]:
age_cat.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [955]:
age_cat.categories[0]

Interval(18, 25, closed='right')

In [956]:
pd.value_counts(age_cat)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In the string representation of an interval, a parenthesis means that the side is open (exclusive), while the square bracket means it is closed (inclusive). You can change which side is closed by passing `right=False`:

In [957]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can override the default interval-based bin labeling by passing a list or array to the `labels` option:

In [958]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to pandas.cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. 

In [959]:
data = np.random.uniform(size=20)

pd.cut(data, 4, precision=2)

[(0.25, 0.5], (0.5, 0.75], (0.25, 0.5], (0.5, 0.75], (0.25, 0.5], ..., (0.0053, 0.25], (0.0053, 0.25], (0.25, 0.5], (0.75, 1.0], (0.75, 1.0]]
Length: 20
Categories (4, interval[float64, right]): [(0.0053, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

The `precision=2` option limits the decimal precision to two digits.

> Cut `ages` into 5 bins instead of 4. Set the labels to `['Youth', 'YoungAdult', 'MiddleAged', 'Senior', 'Elderly']`.

Series containing categorical data have several special methods similar to the `Series.str` specialized string methods. This also provides convenient access to the categories and codes. 

The special accessor attribute `cat` provides access to categorical methods:

In [960]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')

cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [961]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [962]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

Suppose that we know the actual set of categories for this data extends beyond the four values observed in the data. We can use the `set_categories` method to change them:

In [963]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

While it appears that the data is unchanged, the new categories will be reflected in operations that use them. For example, `value_counts` respects the categories, if present:


In [964]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [965]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

Conversely, you can remove categories not in your data by calling `remove_unused_categories`:

In [966]:
cat_s2.cat.remove_unused_categories()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

### Computing Indicator / Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a _dummy_ or _indicator_ matrix. If a column in a DataFrame has `k` distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s.

In [967]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                   "data1": range(6)})

df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [968]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


You can also add a prefix to the columns:

In [969]:
dummies = pd.get_dummies(df["key"], prefix="key")

dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [970]:
np.random.seed(12345)

values = np.random.uniform(size=10)

values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [971]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


> Cut values into 4 bins and create dummy variables.

In [707]:
bins = [0, 0.25, 0.5, 0.75, 1.0]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.25]","(0.25, 0.5]","(0.5, 0.75]","(0.75, 1.0]"
0,0,0,0,1
1,0,1,0,0
2,1,0,0,0
3,1,0,0,0
4,0,0,1,0
5,0,0,1,0
6,0,0,0,1
7,0,0,1,0
8,0,0,1,0
9,0,0,1,0


## Reading / Writing data

Reading and writing data is an essential part of data analysis. Pandas has built-in methods for reading and writing data in different formats. It is also known as data input/output (IO).

### Text files

Pandas features a number of functions for reading (writing) tabular data from (to) text files as DataFrames. You can use `read_csv` to read comma-separated values (CSV) files (which are often exported from spreadsheets or databases). 

For a full list of input/output functions, see [the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/io.html). The optional arguments for these functions may fall into a few categories:

* **Indexing**: You can treat one or more columns as the index of the DataFrame, or you can let pandas create a default integer index.
* **Type inference and data conversion**: This includes the user-defined value conversions and custom list of missing value markers.
* **Datetime parsing**: Includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.
* **Iterating**: Support for iterating over chunks of very large files.
* **Unclean data issues**: Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Some of these functions perform type inference, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, Boolean, or string.

In [972]:
!cat ../data/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [973]:
df = pd.read_csv("../data/ex1.csv")

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row.

In [974]:
!cat ../data/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:

In [975]:
pd.read_csv("../data/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [976]:
pd.read_csv("../data/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the `message` column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named `"message"` using the `index_col` argument:

In [977]:
names = ["a", "b", "c", "d", "message"]

pd.read_csv("../data/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

In [978]:
!cat ../data/ex3.txt

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [979]:
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, `read_csv` infers that the first column should be the DataFrame’s index in this special case.

The file parsing functions have many additional arguments to help you handle the wide variety of exception file formats that occur. For example, you can skip the first, third, and fourth rows of a file with `skiprows`:

In [980]:
!cat ../data/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [981]:
pd.read_csv("../data/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file reading process. Missing data is usually either not present (empty string) or marked by some sentinel (placeholder) value. By default, pandas uses a set of commonly occurring sentinels, such as `NA` and `NULL`:

In [982]:
!cat ../data/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,NULL

In [984]:
result = pd.read_csv("../data/ex5.csv")

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [985]:
result.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,True


`read_csv` has a list of many default NA value representations, but these defaults can be disabled with the `keep_default_na` option:

In [986]:
result2 = pd.read_csv("../data/ex5.csv", keep_default_na=False, na_values=["NA"])

result2

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,NULL


In [987]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


Different NA sentinels can be specified for each column in a dictionary:

In [988]:
sentinels = {"message": ["NULL", "NA"], "something": ["two"]}

pd.read_csv("../data/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


In [989]:
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Using DataFrame’s `to_csv` method, we can write the data out to a comma-separated file:

In [990]:
result.to_csv("../data/out.csv")

In [991]:
!cat ../data/out.csv

,A,B,C
aaa,-0.264438,-1.026059,-0.6195
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


You can also avoid writing the index:

In [992]:
result.to_csv("../data/out.csv", index=False)

In [993]:
!cat ../data/out.csv

A,B,C
-0.264438,-1.026059,-0.6195
0.927272,0.302904,-0.032399
-0.264273,-0.386314,-0.217601
-0.871858,-0.348382,1.100491


> Write only columns A and B to the same file using a parameter in `to_csv`.

In [995]:
mylist = ["A", "B"]
result.to_csv("../data/to_out.csv", index=False, columns=mylist)

### Binary files

`pickle` format is a format that is optimized for reading and writing in Python. It is a format that is specific to Python, and cannot be read by other languages. It is recommended only as a short-term storage format.

In [996]:
result.to_pickle('../data/out.pkl')

In [997]:
data = pd.read_pickle('../data/out.pkl')

data

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


You can read excel file too:

In [998]:
xlsx = pd.ExcelFile("../data/Resaleflatpricesbasedonregistrationdate.xlsx")

xlsx.sheet_names

['2017']

In [999]:
xlsx.parse(sheet_name="2017")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164,Apartment,1992,74 years 05 months,678000


If you are reading multiple sheets in a file, then it is faster to create the `.ExcelFile`, but you can also simply pass the filename to `.read_excel`:

In [1000]:
frame = pd.read_excel("../data/Resaleflatpricesbasedonregistrationdate.xlsx", sheet_name="2017")

frame

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164,Apartment,1992,74 years 05 months,678000


To write pandas data to Excel format, you must first create an `ExcelWriter`, then write data to it using the pandas object's `to_excel` method:

In [1001]:
writer = pd.ExcelWriter('../data/out.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.close()


You can also pass a file path to `to_excel` and avoid the `ExcelWriter`:

In [1002]:
frame.to_excel('../data/out.xlsx')

> Sample 100 rows then write to file: `../data/samples.xlsx`.

In [ ]:
frame.sample(n=100).to_excel('../data/samples.xlsx')

### Databases

You can connect to any databases using the `sqlalchemy` library.

Here, let's connect to the `DuckDB` database we created in unit 4.

In [1004]:
import sqlalchemy as sqla

Let's get the absolute path of the parent (root) directory.

In [1005]:
import os 

parent_dir = os.path.abspath(os.path.pardir)

Then, create the `sqlalchemy` engine to connect to the database.

In [1006]:
engine = sqla.create_engine(f'duckdb:///{parent_dir}/data/unit-1-4.db')

You can read the whole table into a DataFrame:

In [1007]:
df = pd.read_sql('resale_flat_prices_2017', engine)

/home/localuser/miniconda3/envs/pds/lib/python3.10/site-packages/duckdb_engine/__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [1008]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
198037,2025-01,YISHUN,5 ROOM,265,YISHUN ST 22,01 TO 03,122.0,Improved,1986,60 years 11 months,625000.0
198038,2025-01,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,91 years 09 months,730000.0
198039,2025-01,YISHUN,5 ROOM,871,YISHUN ST 81,07 TO 09,122.0,Improved,1988,62 years 04 months,738000.0
198040,2025-01,YISHUN,5 ROOM,851,YISHUN ST 81,10 TO 12,127.0,Improved,1988,62 years 04 months,740000.0


Or SQL query:

In [1009]:
df = pd.read_sql("SELECT * FROM resale_flat_prices_2017", engine)

In [1010]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
198037,2025-01,YISHUN,5 ROOM,265,YISHUN ST 22,01 TO 03,122.0,Improved,1986,60 years 11 months,625000.0
198038,2025-01,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,91 years 09 months,730000.0
198039,2025-01,YISHUN,5 ROOM,871,YISHUN ST 81,07 TO 09,122.0,Improved,1988,62 years 04 months,738000.0
198040,2025-01,YISHUN,5 ROOM,851,YISHUN ST 81,10 TO 12,127.0,Improved,1988,62 years 04 months,740000.0


Write a filtered DataFrame to a new table:

In [1011]:
df_yishun = df[df.town == "YISHUN"]

df_yishun.to_sql("yishun_flat_prices_2017", engine)

ValueError: Table 'yishun_flat_prices_2017' already exists.

In [1012]:
engine.table_names()

/tmp/ipykernel_1366/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['resale_flat_prices_2017', 'yishun_flat_prices_2017']

> Read only flats from `BISHAN` to a new dataframe.
>
> Then, write the dataframe to a new database table `bishan_flat_prices_2017`.

In [1013]:
df_bishan = df[df.town == "BISHAN"]
df_bishan.to_sql("bishan_flat_prices_2017", engine)

-1